In [1]:
import os
import tensorflow as tf
from models import create_model
from data_loader import load_dataset
import datetime

tf.config.run_functions_eagerly(True)
print("Eager execution:", tf.executing_eagerly())

Eager execution: True


In [2]:
%pwd

'/Users/rahulshelke/Documents/Data-Science/Hands-on DL/optimizer_model_comparison'

In [3]:
%ls

README.md                     requirements.txt
__pycache__/                  results/
data_loader.py                run_experiments.py
models.py                     train.py
optimizers_comparision.ipynb


## Load Datasets

In [4]:
def load_dataset(name):
    if name == 'mnist':
        data = tf.keras.datasets.mnist
    elif name == 'fashion_mnist':
        data = tf.keras.datasets.fashion_mnist
    elif name == 'cifar10':
        data = tf.keras.datasets.cifar10
    else:
        raise ValueError("Unsupported dataset")

    (x_train, y_train), (x_test, y_test) = data.load_data()

    if name == 'cifar10':
        x_train = x_train.astype('float32') / 255.0
        x_test = x_test.astype('float32') / 255.0
    else:
        x_train = x_train.reshape(-1, 28, 28, 1).astype('float32') / 255.0
        x_test = x_test.reshape(-1, 28, 28, 1).astype('float32') / 255.0

    return (x_train, y_train), (x_test, y_test)

## Models

In [5]:
def create_model(size='small', input_shape=(28, 28, 1), num_classes=10):
    model = tf.keras.Sequential()
    model.add(tf.keras.Input(shape=input_shape))

    if size == 'small':
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(64, activation='relu'))

    elif size == 'medium':
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(128, activation='relu'))

    elif size == 'large':
        model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu'))
        model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
        model.add(tf.keras.layers.MaxPooling2D((2, 2)))
        model.add(tf.keras.layers.Flatten())
        model.add(tf.keras.layers.Dense(256, activation='relu'))
        model.add(tf.keras.layers.Dropout(0.5))

    model.add(tf.keras.layers.Dense(num_classes, activation='softmax'))
    return model

## Training Loop

In [6]:
def train_model(model_size, dataset_name, optimizer, epochs=5, batch_size=64):
    (x_train, y_train), (x_test, y_test) = load_dataset(dataset_name)
    input_shape = x_train.shape[1:]
    num_classes = 10

    model = create_model(model_size, input_shape, num_classes)

    model.compile(optimizer=optimizer,
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])

    log_dir = f"results/{dataset_name}/{model_size}/{type(optimizer).__name__}/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
    os.makedirs(log_dir, exist_ok=True)

    callbacks = [tf.keras.callbacks.TensorBoard(log_dir=log_dir)]

    history = model.fit(
        x_train, y_train,
        validation_data=(x_test, y_test),
        epochs=epochs,
        batch_size=batch_size,
        callbacks=callbacks,
        verbose=2
    )

    return history


In [8]:
datasets = ['mnist', 
# 'fashion_mnist', 'cifar10'
]
model_sizes = ['small', 'medium', 'large']
optimizers = {
    'SGD': tf.keras.optimizers.SGD(),
    'Adam': tf.keras.optimizers.Adam(),
    'RMSprop': tf.keras.optimizers.RMSprop()
}

In [9]:
for dataset in datasets:
    for size in model_sizes:
        for name, opt in optimizers.items():
            print(f"\n--- Training {size} model on {dataset} with {name} ---")
            train_model(model_size=size, dataset_name=dataset, optimizer=opt)



--- Training small model on mnist with SGD ---
Epoch 1/5


/opt/anaconda3/envs/ml-macos-m2/lib/python3.10/site-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


938/938 - 13s - 14ms/step - accuracy: 0.7749 - loss: 0.9092 - val_accuracy: 0.8823 - val_loss: 0.4626
Epoch 2/5
938/938 - 13s - 14ms/step - accuracy: 0.8874 - loss: 0.4205 - val_accuracy: 0.9024 - val_loss: 0.3568
Epoch 3/5
938/938 - 12s - 13ms/step - accuracy: 0.9018 - loss: 0.3525 - val_accuracy: 0.9107 - val_loss: 0.3184
Epoch 4/5
938/938 - 13s - 14ms/step - accuracy: 0.9099 - loss: 0.3186 - val_accuracy: 0.9165 - val_loss: 0.2933
Epoch 5/5
938/938 - 13s - 14ms/step - accuracy: 0.9161 - loss: 0.2965 - val_accuracy: 0.9217 - val_loss: 0.2752

--- Training small model on mnist with Adam ---
Epoch 1/5
938/938 - 23s - 24ms/step - accuracy: 0.9035 - loss: 0.3446 - val_accuracy: 0.9426 - val_loss: 0.1967
Epoch 2/5
938/938 - 23s - 24ms/step - accuracy: 0.9532 - loss: 0.1633 - val_accuracy: 0.9601 - val_loss: 0.1355
Epoch 3/5
938/938 - 23s - 24ms/step - accuracy: 0.9653 - loss: 0.1202 - val_accuracy: 0.9679 - val_loss: 0.1112
Epoch 4/5
938/938 - 23s - 25ms/step - accuracy: 0.9721 - loss: 0.

ValueError: Unknown variable: <Variable path=sequential_3/conv2d/kernel, shape=(3, 3, 1, 32), dtype=float32, value=[[[[-0.0365007   0.06048135 -0.07531459  0.10370916 -0.03047556
    -0.05668736  0.03010792 -0.00141899 -0.04018178  0.06333803
     0.09014285  0.07875232 -0.10099072 -0.05712081  0.10191517
    -0.04997095 -0.03953873 -0.01535632 -0.09141612  0.08703452
     0.06845456  0.00731577 -0.13980752 -0.05907621  0.08032005
     0.13709901  0.05532143 -0.00927801  0.07934175  0.02873382
     0.02481079 -0.05120631]]

  [[ 0.09291896  0.08649303 -0.04074316 -0.02634504  0.06421356
     0.12940721  0.00873372 -0.01456688  0.11392339  0.09660241
    -0.05467994 -0.13137269 -0.06201024 -0.00159287 -0.12166966
    -0.01512639 -0.05282332  0.04221871  0.06175859  0.01525134
     0.01895292  0.09232108 -0.01249762 -0.03818327  0.13394
     0.01080692 -0.09308334  0.01048559  0.12741838  0.09904006
     0.04582287 -0.03895862]]

  [[ 0.13946585  0.03910677 -0.05318601  0.07292618 -0.106